In [12]:
# use venv ~opensim/bin/python

import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm


In [13]:
# get random index for IMU sensor
def sampleSensor(sensorName):
    index = random.randint(0, 49)
    return sensorName if index == 0 else sensorName + str(index)

In [14]:
def scaleDataInstance(data):
    for i in range(data.shape[1]):
        data[:, i] = data[:, i] / max(abs(data[:, i]))
    return data


# take max abs value of each channel
# divide all values by max value

In [15]:
def removeZeros(data):
    return np.delete(data, [0, 16], axis=1)

In [16]:
def createIMUDataInstance(quaternion_data, velocity_data, acceleration_data, orientation_only = False):
    # Get indices for sensors to use in this data instance
    humerus_l_imu_tag = "humerus_l_imu"
    humerus_r_imu_tag = "humerus_r_imu"
    ulna_l_imu_tag = "ulna_l_imu"
    ulna_r_imu_tag = "ulna_r_imu"
    tags = [humerus_l_imu_tag, humerus_r_imu_tag, ulna_l_imu_tag, ulna_r_imu_tag]

    concatenated = []
    for tag in tags:
        raw_q = quaternion_data[tag].values
        raw_v = velocity_data[tag].values
        raw_a = acceleration_data[tag].values

        if not orientation_only:
            total = np.zeros([raw_q.shape[0], 10])
            for i in range(raw_q.shape[0]):
                total[i, :4] = np.array(raw_q[i].split(',')).astype(float)
                total[i, 4:7] = np.array(raw_v[i].split(',')).astype(float)
                total[i, 7:] = np.array(raw_a[i].split(',')).astype(float)
        else:
            total = np.zeros([raw_q.shape[0], 4])
            for i in range(raw_q.shape[0]):
                total[i, :4] = np.array(raw_q[i].split(',')).astype(float)
        
        concatenated.append(total)

    concatenated_data = np.concatenate(concatenated, axis=1)
    return concatenated_data


In [17]:
def formatDataInstance(kinematics_data, quaternion_data, velocity_data, acceleration_data):

    # get imu data at time index
    new_IMU_instance = createIMUDataInstance(quaternion_data, velocity_data, acceleration_data, orientation_only=True)
    right_arm_kinematics = kinematics_data.values[:, 26:31]
    left_arm_kinematics = kinematics_data.values[:, 33:38]
    right_leg_kinematics = kinematics_data.values[:, 7:13]
    left_leg_kinematics = kinematics_data.values[:, 15:21]
    torso_kinematics = np.concatenate([kinematics_data.values[:, 1:7],  kinematics_data.values[:, 23:26]], axis=1)
    new_kinematics_instance = np.concatenate([left_arm_kinematics, 
                                              right_arm_kinematics,
                                            #   left_leg_kinematics,
                                            #   right_leg_kinematics,
                                              torso_kinematics], axis = 1)

    print("Kinematics Dimensions: ", new_kinematics_instance.shape)
    print("IMU Dimensions: ", new_IMU_instance.shape)
    # concatenate and return
    new_data_instance = np.concatenate([new_kinematics_instance, new_IMU_instance], axis=1)
    print("Concatenated Dimensions: ", new_data_instance.shape)
    return new_data_instance

In [18]:
def augment_temporal_minus_one(data):
    r = data.shape[0]
    c = data.shape[1]

    res = np.zeros((r-1, c*2))
    for j in range(c):
        res[:, j*2] = data[1:, j]
        res[:, j*2+1] = data[0:r-1, j]
    
    return res

In [19]:
def augment_temporal_plus_one(data):
    r = data.shape[0]
    c = data.shape[1]

    res = np.zeros((r-1, c*2))
    for j in range(c):
        res[:, j*2] = data[0:r-1, j]
        res[:, j*2+1] = data[1:, j]
    return res

In [20]:
def mask(data):
    r = data.shape[0]
    c = data.shape[1]
    res = np.copy(data)

    for j in range(0, 38, 2):
        res[:, j] = -2
    return np.vstack((data, res))

In [22]:
def createDataset(trial_num, trial_type):

    filepath = "/home/franklin/Research"
    # Local: /Users/FranklinZhao/Research
    # SSH: /home/franklin/Research



    # load IMU data, non-filtered
    # quaternion_path = f"{filepath}/Dataset/vIMU/Experimental/{trial_type}_{trial_num}/Rajapogal_FullBody_calibrated_orientations.sto"
    # velocity_path = f"{filepath}/Dataset/vIMU/Experimental/{trial_type}_{trial_num}/Rajapogal_FullBody_calibrated_angular_velocity.sto"
    # acceleration_path = f"{filepath}/Dataset/vIMU/Experimental/{trial_type}_{trial_num}/Rajapogal_FullBody_calibrated_linear_accelerations.sto"

    # load IMU data, filtered
    quaternion_path = f"{filepath}/Dataset/vIMU/Experimental/{trial_type}_{trial_num}/filtered_orientations.sto"
    velocity_path = f"{filepath}/Dataset/vIMU/Experimental/{trial_type}_{trial_num}/filtered_angular_velocity.sto"
    acceleration_path = f"{filepath}/Dataset/vIMU/Experimental/{trial_type}_{trial_num}/filtered_linear_accelerations.sto"

    # read IMU data into dataframes
    quaternion_data = pd.read_csv(filepath_or_buffer=quaternion_path, sep='\t', header=4)
    velocity_data = pd.read_csv(filepath_or_buffer=velocity_path, sep='\t', header=4)
    acceleration_data = pd.read_csv(filepath_or_buffer=acceleration_path, sep='\t', header=4)

    # load kinematics data
    kinematics_path = f"{filepath}/Dataset/Motion/Experimental/{trial_type}_0{trial_num}.mot"

    # read kinematics data into dataframes
    kinematics_data = pd.read_csv(filepath_or_buffer=kinematics_path, header=8, skipinitialspace=True, sep='\t')    

    # create data instance
    new_data_instance = formatDataInstance(kinematics_data, quaternion_data, velocity_data, acceleration_data)
    # scale data instance
    new_data_instance = scaleDataInstance(new_data_instance)
    # new_data_instance = augment_temporal_minus_one(new_data_instance)
    # new_data_instance = mask(new_data_instance)
    print(new_data_instance.shape)

    # save data instance
    df = pd.DataFrame(new_data_instance)
    file_path = f'{filepath}/data_instance_{trial_num}.csv'
    df.to_csv(file_path, index=False, header=False)
    print("saved!")

In [23]:
# trial data
trial_type = "tug"
for i in range(1, 4):
    print(f"Creating Dataset {i}")
    createDataset(i, trial_type)

Creating Dataset 1
Kinematics Dimensions:  (7510, 19)
IMU Dimensions:  (7510, 16)
Concatenated Dimensions:  (7510, 35)
(7510, 35)
saved!
Creating Dataset 2
Kinematics Dimensions:  (5405, 19)
IMU Dimensions:  (5405, 16)
Concatenated Dimensions:  (5405, 35)
(5405, 35)
saved!
Creating Dataset 3
Kinematics Dimensions:  (5203, 19)
IMU Dimensions:  (5203, 16)
Concatenated Dimensions:  (5203, 35)
(5203, 35)
saved!
